安裝套件

In [ ]:
! pip install selenium
! pip install chromedriver-autoinstaller-fix

Selenium

In [ ]:
from math import floor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import chromedriver_autoinstaller
from time import sleep

# 安裝或更新 ChromeDriver
chromedriver_autoinstaller.install()

# 使用 with 語句管理 WebDriver
with webdriver.Chrome() as driver:
    driver.get("https://stats.moe.gov.tw/bcode/")

    # 選擇搜尋條件並按下查詢
    driver.find_element(By.ID, "RadioButtonList2_2").click()
    SelectElem = driver.find_element(By.NAME, "DropDownList1")
    Select(SelectElem).select_by_value("01")
    driver.find_element(By.ID, "Button1").click()

    # 先建立一個陣列來放要存進csv的資料
    table = [["學年度", "學校", "體系別", "設立別", "學校", "代碼", "學校名稱", "領域代碼/名稱", "學門代碼/名稱",
              "學類代碼/名稱", "系學類代碼/名稱", "科系代碼", "科系名稱", "日間/進修別", "等級別", "學生數總計", "男學生數", "女學生數"]]

    totalPage = floor(int(driver.find_element(
        By.ID, 'RecordCnt').find_element(By.TAG_NAME, "font").text)/10+1)

    print("totalPage", totalPage)

    for currentpage in range(1, totalPage + 1):
        for i, row in enumerate(driver.find_elements(By.CLASS_NAME, "GridViewRowStyle")):
            rowData = []
            for column in row.find_elements(By.TAG_NAME, "td"):
                rowData.append(column.text)
            table.append(rowData)

        # 換頁確認
        if currentpage < totalPage:
            try:
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, 'BtnNext'))).click()
                print("currentpage", currentpage)
                # 避免伺服器壓力過大
                sleep(1)
            except Exception as e:
                print(f"換頁失敗：{str(e)}")

# 等待 5 秒再關閉瀏覽器
sleep(5)

In [ ]:
import csv

# 假設前面的程式碼已經執行過，得到了 table 這個二維列表

with open('result.csv', 'w', newline='', encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile)

    # 寫入二維表格
    writer.writerows(table)

print("CSV 檔案已成功寫入。")

Beautiful soup

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import csv
from time import sleep

# 安裝或更新 ChromeDriver
chromedriver_autoinstaller.install()

# 讓webdriver開啟我們要的網站
driver = webdriver.Chrome()
driver.get("https://ithelp.ithome.com.tw/")
driver.fullscreen_window()

# 等待一些時間，讓網頁充分加載
sleep(2)

# 獲取網頁原始碼
page_html = driver.page_source

# 關閉瀏覽器
driver.quit()

# 使用 BeautifulSoup 解析網頁
soup = BeautifulSoup(page_html, 'html.parser')

# 提取問答區的相關資訊
question_columns = soup.select('.qa-list')

result = [['喜歡數', '回應數', '瀏覽數', '標題', '時間', '提問人']]
for question_column in question_columns:
    like_count = question_column.select('.qa-condition__count')[0].text
    response_count = question_column.select('.qa-condition__count')[1].text
    view_count = question_column.select('.qa-condition__count')[2].text
    topic = question_column.select('.qa-list__title>a')[0].text
    time = question_column.select('.qa-list__info-time')[0].text
    people = question_column.select('.qa-list__info-link')[0].text
    people = people.replace('\n', '').replace(' ', '')
    col = [like_count, response_count, view_count, topic, time, people]
    result.append(col)

# 輸出結果到 CSV 檔案
with open('resultbs4.csv', 'w', newline='', encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile)

    # 寫入二維表格
    writer.writerows(result)

print(result)